## To Do List

# User Parameters

#Prep Data

##Mount Drives, Import Packages, Load Files

Import Packages

In [1]:
!pip install xlsxwriter
#!pip install sharepy
!pip install google_trans_new
!pip install pyxlsb --upgrade
!pip install git+https://github.com/applio/pyxlsb.git@enh_basic_writing --upgrade
!pip install gspread
!pip install oauth2client
!pip install df2gspread
!pip install pandas --upgrade
!pip install pantab

import pantab
import pandas as pd
import time
import scipy.stats as stats
import numpy as np
import datetime as dt
from pandas import ExcelWriter #for writing xlsx
from pyxlsb import open_workbook #for writing to xlsb
from datetime import datetime
import sharepy as sharepy #for loading from sharepoint
#from google.colab import files #for accessing google colab files systemically
from google_trans_new import google_translator #for language translation
from pprint import pprint #for printing pretty
#below for writing to google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

pd.set_option('mode.chained_assignment', None)
print('libraries read!')

  Running command git clone -q https://github.com/applio/pyxlsb.git 'C:\Users\William McDonald\AppData\Local\Temp\pip-req-build-briuz1lu'



  Cloning https://github.com/applio/pyxlsb.git (to revision enh_basic_writing) to c:\users\william mcdonald\appdata\local\temp\pip-req-build-briuz1lu
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


libraries read!


In [2]:
#Sharepoint load of files -- WIP

#s = sharepy.connect("https://sncfl.sharepoint.com")
#r = s.get("https://sncfl.sharepoint.com/:f:/r/sites/msteams_e7402d/Shared%20Documents/DATA%20MIRABELLE%20-%20OUT/TRAINS?csf=1&web=1&e=wMMSIz")
#r = s.get("https://sncfl.sharepoint.com/sites/msteams_e7402d/_layouts/15/WopiFrame2.aspx?sourcedoc={0f722845-46fe-48cb-89f1-113a7e76c1a5}&action=edit")

In [3]:
#WM Directory
source_dir = r'C:\Users\William McDonald\Google Drive\LCL\CFL_Thesis\Data'
output_dir = r'C:\Users\William McDonald\Google Drive\LCL\CFL_Thesis\writeup'
#JP Directory

#WZ Directory

Load Files

In [4]:
train = pd.read_excel(source_dir + r'\TRAIN.xlsx') #joined train_data

In [5]:
train_etape = pd.read_excel(source_dir + r'\TRAIN_ETAPE.xlsx') #joined train_data

In [6]:
train_jalon = pd.read_excel(source_dir + r'\TRAIN_JALON.xlsx') #joined station_data

In [7]:
train_lot = pd.read_excel(source_dir + r'\TRAIN_LOT.xlsx') #joined train_data

In [8]:
train_wagon = pd.read_excel(source_dir + r'\TRAIN_WAGON.xlsx') #joined wagon_data

In [9]:
train_position = pd.read_excel(source_dir + r'\TRAIN_POSITION.xlsx')

In [10]:
wagon_position = pd.read_csv(source_dir + r'\WAGON_POSITION.csv')

In [11]:
wagon = pd.read_excel(source_dir + r'\WAGON.xlsx') #joined wagon_data

In [12]:
wagon_modelle = pd.read_excel(source_dir + r'\WAGON.xlsx', sheet_name=1) #joined wagon_data

In [13]:
wagon_capacite = pd.read_excel(source_dir + r'\WAGON_CAPACITE.xlsx') #joined wagon_data

In [14]:
terminal = pd.read_excel(source_dir + r'\TERMINAL.xlsx')

In [15]:
station = pd.read_excel(source_dir + r'\STATION_WM.xlsx') #joined station_data

In [16]:
incident = pd.read_excel(source_dir + r'\INCIDENTS.xlsx')

In [17]:
incident_concerne = pd.read_excel(source_dir + r'\INCIDENT_CONCERNE.xlsx')

In [18]:
train_translation = pd.read_excel(source_dir + r'\TRAIN_Translation.xlsx')

In [19]:
data_files = [train, train_etape, train_jalon, train_lot,train_wagon, train_position, wagon_position, wagon, wagon_modelle, wagon_capacite, terminal, station, incident, incident_concerne, train_translation]

In [20]:
for d in data_files:
    d = (d.drop_duplicates().reset_index(drop=True))

Original Data Shapes

In [21]:
print('File Shapes (row count, column count)')
print('train: ',train.shape, 'Key: IDTRAIN')
print('train_etape: ', train_etape.shape, 'Key: IDTRAIN_LOT')
print('train_jalon: ',train_jalon.shape, 'Key: IDTRAIN_JALON')
print('train_lot: ',train_lot.shape, 'Key: IDTRAIN_LOT')
print('train_wagon: ',train_wagon.shape)
print('train_position: ',train_position.shape)
print('wagon_position: ',wagon_position.shape)
print('wagons: ',wagon.shape)
print('wagon_modelle: ',wagon_modelle.shape)
print('wagon_capacite: ',wagon_capacite.shape)
print('terminal: ',terminal.shape)
print('station: ',station.shape, 'Key:IDGARE')
print('incidents: ',incident.shape)
print('incident_concerne: ',incident_concerne.shape)
print('train_translation: ',train_translation.shape)

File Shapes (row count, column count)
train:  (14215, 19) Key: IDTRAIN
train_etape:  (147208, 13) Key: IDTRAIN_LOT
train_jalon:  (72531, 27) Key: IDTRAIN_JALON
train_lot:  (14692, 45) Key: IDTRAIN_LOT
train_wagon:  (226998, 10)
train_position:  (25343, 5)
wagon_position:  (2528599, 1)
wagons:  (523, 33)
wagon_modelle:  (14, 43)
wagon_capacite:  (29, 10)
terminal:  (19, 12)
station:  (7175, 19) Key:IDGARE
incidents:  (5875, 17)
incident_concerne:  (59773, 17)
train_translation:  (14414, 3)


Remove Duplicates from Keys

In [22]:
train = train.drop_duplicates(subset=['IDTRAIN'], keep='first')
train_etape = train_etape.drop_duplicates(subset=['IDTRAIN_LOT'], keep='first')
train_lot = train_lot.drop_duplicates(subset=['IDTRAIN_LOT'], keep='first')
station = station.drop_duplicates(subset=['IDGARE'], keep='first')
train_jalon = train_jalon.drop_duplicates(subset=['IDTRAIN_JALON'], keep='first')

## Translate Incident Comments to English

In [23]:
#translate new comments that need it
common = pd.merge(train, train_translation, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='inner')
new_translate = train[~train['IDTRAIN'].isin(common['IDTRAIN'])]

if len(new_translate['IDTRAIN']) > 0:
    translator = google_translator()
    comments_to_translate_frame = new_translate[['IDTRAIN','Commentaire']]
    comments_to_translate_frame = comments_to_translate_frame.reset_index(drop=True)
    comments_to_translate_list = new_translate['Commentaire'].tolist()
    item_count = 0
    translated_comments = []
    for i in comments_to_translate_list:
        translated_comments.append(translator.translate(i,lang_tgt='en'))
        item_count += 1
        print('Translated ' + str(item_count) + ' comments. Translated item was ' + str(i) + '.')
        time.sleep(2)
    translated_comments = pd.DataFrame(data=translated_comments, columns=['English_Comments'])
    translated_comments_frame = pd.merge(comments_to_translate_frame, translated_comments,left_index=True)
    train_translation = pd.merge(train_translation, translated_comments_frame, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='outer')
    writer = pd.ExcelWriter(source_dir + r'/TRAIN_translation.xlsx', engine='xlsxwriter')
    train_translation.to_excel(writer, index=False)
    writer.save()

train = pd.merge(train,train_translation, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='left')

##Joins for Core Datasets

Wagons

In [24]:
wagon_data = pd.merge(train_wagon, wagon, left_on= ['IDWAGON'], right_on= ['IDWAGON'], how = 'left', indicator='wagon_join_01', suffixes=('_train_wagon','_wagon'))
wagon_data = pd.merge(wagon_data, wagon_modelle, left_on=  ['IDWAGON_MODELE_train_wagon'], right_on= ['IDWAGON_MODELE'], how = 'left', indicator='wagon_join_02', suffixes=('','_wagon_modelle'))

Stations

In [25]:
station_data = pd.merge(train_jalon, station, left_on= ['IDGARE'], right_on= ['IDGARE'], how='left', indicator='station_join_01', suffixes=('_train_jalon','_station'))

Trains

In [26]:
origin = station_data[station_data['Jalon_num']==1]
destination = station_data[station_data['Jalon_num']==9999]
train_od = pd.merge(origin,destination, left_on= ['IDTRAIN'], right_on= ['IDTRAIN'], how='inner', indicator='train_od_join_01', suffixes=('_origin','_destination'))

train_data = pd.merge(train_lot, train, left_on=  ['IDTRAIN'], right_on= ['IDTRAIN'], how = 'left', indicator='trains_join_01', suffixes=('_train_lot','_train'))
train_data = pd.merge(train_data, train_etape, left_on=  ['IDTRAIN_LOT'], right_on= ['IDTRAIN_LOT'], how = 'left', indicator='trains_join_02', suffixes=('','_train_etape'))
train_data = train_data.drop_duplicates(subset=['IDTRAIN'], keep='first')
train_data = pd.merge(train_od,train_data, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='left', indicator='trains_join_03', suffixes=('_train_od',''))

Core Files

In [27]:
print('Initial Data Shapes')
print('train_data: ', train_data.shape)
print('wagon_data: ', wagon_data.shape)
print('station_data: ', station_data.shape)
#print('incident_data: ',incident_data.shape)

Initial Data Shapes
train_data:  (14215, 171)
wagon_data:  (226998, 87)
station_data:  (72531, 46)


##Remove Unnecessary Fields

In [28]:
#remove fields with only null
train_data = train_data.dropna(axis=1, how='all')
wagon_data = wagon_data.dropna(axis=1, how='all')
station_data = station_data.dropna(axis=1, how='all')

#remove fields with no unique data
def remove_constant_value_features(df):
    return [e for e in df.columns if df[e].nunique() == 1]

drop_col_train_data = remove_constant_value_features(train_data)
drop_col_wagon_data = remove_constant_value_features(wagon_data)
drop_col_station_data = remove_constant_value_features(station_data)

train_data = train_data.drop(labels=drop_col_train_data, axis=1)
wagon_data = wagon_data.drop(labels=drop_col_wagon_data, axis=1)
station_data = station_data.drop(labels=drop_col_station_data, axis=1)

#remove fields with identical values to another field
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

duplicate_train_data_fields = getDuplicateColumns(train_data)
duplicate_wagon_data_fields = getDuplicateColumns(wagon_data)
duplicate_station_data_fields = getDuplicateColumns(station_data)

train_data = train_data.drop(labels=duplicate_train_data_fields, axis=1)
wagon_data = wagon_data.drop(labels=duplicate_wagon_data_fields, axis=1)
station_data = station_data.drop(labels=duplicate_station_data_fields, axis=1)

##Select Relevant Fields

train_data Cleanup

In [29]:
train_data_fields = ['IDTRAIN'
                        ,'IDTRAIN_LOT'
                        ,'IDTRAIN_JALON_origin'
                        ,'IDGARE_origin'
                        ,'Code_origin'
                        ,'IDGARE_Parent_origin'
                        ,'IDTRAIN_JALON_destination'
                        ,'IDGARE_destination'
                        ,'Code_destination'
                        ,'IDGARE_Parent_destination'
                        ,'Train_num'
                        ,'IDTRAIN_Parent'
                        ,'IDTRAIN_ETAPE'
                        ,'NomComplet_origin'
                        ,'Code_pays_station_origin'
                        ,'Latitude_station_origin'
                        ,'Longitude_station_origin'
                        ,'DHT_Depart_origin'
                        ,'DHR_Depart_origin'
                        ,'NomComplet_destination'
                        ,'Code_pays_station_destination'
                        ,'Latitude_station_destination'
                        ,'Longitude_station_destination'
                        ,'DHT_Arrivee_destination'
                        ,'DHR_Arrivee_destination'
                        ,'Distance'
                        ,'Incoterm'
                        ,'Capacite_TEU'
                        ,'Nbre_TEU'
                        ,'Longueur_maxi'
                        ,'Longueur_totale'
                        ,'Poids_total'
                        ,'Motif_suppression_train'
                        ,'Commentaire'
                        ,'English_Comments'
                        ]

train_data_rename = ['IDTRAIN'
                        ,'IDTRAIN_LOT'
                        ,'IDTRAIN_JALON_origin'
                        ,'IDGARE_origin'
                        ,'Code_origin'
                        ,'IDGARE_Parent_origin'
                        ,'IDTRAIN_JALON_destination'
                        ,'IDGARE_destination'
                        ,'Code_destination'
                        ,'IDGARE_Parent_destination'
                        ,'Train_num'
                        ,'IDTRAIN_Parent'
                        ,'IDTRAIN_ETAPE'
                        ,'Origin_Station'
                        ,'Origin_Country'
                        ,'Origin_Latitude'
                        ,'Origin_Longitude'
                        ,'Planned_Departure'
                        ,'Actual_Departure'
                        ,'Destination_Station'
                        ,'Destination_Country'
                        ,'Destination_Latitude'
                        ,'Destination_Longitude'
                        ,'Planned_Arrival'
                        ,'Actual_Arrival'
                        ,'Journey_Distance'
                        ,'Incoterm'
                        ,'Max_TEU'
                        ,'TEU_Count'
                        ,'Max_Length'
                        ,'Train_Length'
                        ,'Train_Weight'
                        ,'Cancellation_Reason'
                        ,'French_Cancel_Comments'
                        ,'English_Cancel_Comments'
                        ]

train_data_remainder = train_data.drop(columns=train_data_fields,errors='ignore')
train_data = train_data[train_data_fields]
train_data.columns=train_data_rename
train_data = train_data.sort_values(by='Planned_Departure')
train_data = train_data.reset_index(drop=True)

train_data

,IDTRAIN,IDTRAIN_LOT,IDTRAIN_JALON_origin,IDGARE_origin,Code_origin,IDGARE_Parent_origin,IDTRAIN_JALON_destination,IDGARE_destination,Code_destination,IDGARE_Parent_destination,...,Journey_Distance,Incoterm,Max_TEU,TEU_Count,Max_Length,Train_Length,Train_Weight,Cancellation_Reason,French_Cancel_Comments,English_Cancel_Comments
0,269,269,3619,84831,14108,0.0,3620,80248,30100,25174.0,...,99,EXW,NaN,NaN,400,NaN,NaN,Période de tests,NaN,male
1,259,259,3539,80248,30100,25174.0,3540,84831,14108,0.0,...,99,EXW,NaN,NaN,400,NaN,NaN,Période de tests,NaN,male
2,362,362,4363,80248,30100,25174.0,4370,85346,3471,0.0,...,1137,EXW,NaN,NaN,548,NaN,NaN,Période de tests,NaN,male
3,274,274,3659,84831,14108,0.0,3660,80248,30100,25174.0,...,99,EXW,NaN,NaN,400,NaN,NaN,Période de tests,NaN,male
4,264,264,3579,80248,30100,25174.0,3580,84831,14108,0.0,...,99,EXW,NaN,NaN,400,NaN,NaN,Période de tests,NaN,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14210,20739,22004,107866,85346,3471,0.0,107873,80248,30100,25174.0,...,1137,EXW,72.0,0.0,548,544.0,560000.0,NaN,NaN,male
14211,19031,20296,95884,80248,30100,25174.0,95885,92363,78433,0.0,...,1045,-1,90.0,0.0,1000,680.0,767000.0,NaN,NaN,male
14212,18163,19428,94148,92363,78433,0.0,94149,80248,30100,25174.0,...,1045,-1,90.0,0.0,1000,680.0,767000.0,NaN,NaN,male
14213,21301,22566,112480,80248,30100,25174.0,112487,85346,3471,0.0,...,1137,EXW,72.0,0.0,548,544.0,560000.0,NaN,NaN,male


wagon_data Cleanup

In [30]:
wagon_data_fields = ['IDTRAIN_LOT'
                    ,'IDTRAIN_WAGON'
                    ,'IDWAGON'
                    ,'Modele_wagon'
                    ,'Code_wagon'
                    ,'IDGARE_actuelle'
                    ,'Num_Position'
                    ,'Type_wagon'
                    ,'Vitesse_maxi'
                    ,'Tare_wagon_modelle'
                    ,'Tare'
                    ,'Date_mise_circulation']

wagon_data_rename = ['IDTRAIN_LOT'
                    ,'IDTRAIN_WAGON'
                    ,'IDWAGON'
                    ,'Modele_wagon'
                    ,'Code_wagon'
                    ,'IDGARE_actuelle'
                    ,'Wagon_Order'
                    ,'Wagon_Type'
                    ,'Max_Speed'
                    ,'Wagon_Model_Tare_Weight'
                    ,'Tare_Weight'
                    ,'Wagon_Start_Date' ]

wagon_data_remainder = wagon_data.drop(columns=wagon_data_fields,errors='ignore')
wagon_data = wagon_data[wagon_data_fields]
wagon_data.columns = wagon_data_rename

wagon_data

,IDTRAIN_LOT,IDTRAIN_WAGON,IDWAGON,Modele_wagon,Code_wagon,IDGARE_actuelle,Wagon_Order,Wagon_Type,Max_Speed,Wagon_Model_Tare_Weight,Tare_Weight,Wagon_Start_Date
0,7138,138270,NaN,T3000e NIK C30,NaN,NaN,0,DOUBLE,120,35000,NaN,NaN
1,7138,138271,NaN,60',NaN,NaN,1,SIMPLE,120,19800,NaN,NaN
2,7138,138272,NaN,60',NaN,NaN,2,SIMPLE,120,19800,NaN,NaN
3,7138,138273,NaN,60',NaN,NaN,3,SIMPLE,120,19800,NaN,NaN
4,7138,138274,NaN,60',NaN,NaN,4,SIMPLE,120,19800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
226993,22078,578010,170.0,90',33 88 4962 568-0,82517.0,11,DOUBLE,120,27200,27160.0,NaN
226994,22078,578011,144.0,90',33 88 4961 185-4,82517.0,12,DOUBLE,120,27200,27200.0,NaN
226995,22078,578012,165.0,90',33 88 4962 454-3,82517.0,13,DOUBLE,120,27200,27160.0,NaN
226996,22078,578013,167.0,90',33 88 4962 514-4,82517.0,14,DOUBLE,120,27200,27160.0,NaN


station_data Cleanup

In [31]:
station_data_fields = ['IDTRAIN'
                    ,'IDGARE'
                    ,'IDTRAIN_JALON'
                    ,'Jalon_num'
                    ,'NomComplet'
                    ,'Code_pays_station'
                    ,'Latitude_station'
                    ,'Longitude_station'
                    ,'DHT_Arrivee'
                    ,'DHR_Arrivee'
                    ,'DHT_Depart'
                    ,'DHR_Depart']

station_data_rename = ['IDTRAIN'
                    ,'IDGARE'
                    ,'IDTRAIN_JALON'
                    ,'Station_Order'
                    ,'Station_Name'
                    ,'Station_Country'
                    ,'Station_Latitude'
                    ,'Station_Longitude'
                    ,'Planned_Arrival'
                    ,'Actual_Arrival'
                    ,'Planned_Departure'
                    ,'Actual_Departure']

station_data_remainder = station_data.drop(columns=station_data_fields,errors='ignore')
station_data = station_data[station_data_fields]
station_data.columns = station_data_rename
station_data = station_data.sort_values(by=['IDTRAIN','Station_Order'])
station_data = station_data.reset_index(drop=True)

station_data

,IDTRAIN,IDGARE,IDTRAIN_JALON,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Planned_Arrival,Actual_Arrival,Planned_Departure,Actual_Departure
0,255,80248,3507,1,Bettembourg-Container-Terminal,LU,49.493889,6.108889,NaN,NaN,2019-03-04 01:50:00.000,NaN
1,255,85792,3510,2,Bettembourg-frontière,LU,49.467859,6.109153,2019-03-04 04:50:00.000,NaN,2019-03-04 04:50:00.000,NaN
2,255,87910,3509,3,Hagondange,FR,49.245710,6.162933,2019-03-04 05:30:00.000,NaN,2019-03-04 05:30:00.000,NaN
3,255,87942,3511,4,Woippy,FR,49.149512,6.156061,2019-03-04 05:40:00.000,NaN,2019-03-04 05:40:00.000,NaN
4,255,87939,3512,5,Metz-Chambiere,FR,49.126336,6.159898,2019-03-04 05:55:00.000,NaN,2019-03-04 05:55:00.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
72526,22417,67920,120915,7,Ehrang,DE,49.802000,6.686000,2021-04-10 08:43:00.000,NaN,2021-04-10 09:14:00.000,NaN
72527,22417,85881,120918,8,Apach-Frontiere,FR,49.459076,6.372680,2021-04-10 11:49:00.000,NaN,2021-04-10 11:50:00.000,NaN
72528,22417,87935,120919,9,Metz-Sablon,FR,49.096407,6.166320,2021-04-10 12:48:00.000,NaN,2021-04-10 12:55:00.000,NaN
72529,22417,85791,120916,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,2021-04-10 13:54:00.000,NaN,2021-04-10 13:55:00.000,NaN


Rename Values Stations

In [32]:
train_data.loc[train_data['Origin_Station'] == 'Antwerpen-Buitenschoor-Combinant', 'Origin_Station'] = 'Antwerp'
train_data.loc[train_data['Origin_Station'] == 'Barcelona can Tunis', 'Origin_Station'] = 'Barcelona'
train_data.loc[train_data['Origin_Station'] == 'Bettembourg-Container-Terminal', 'Origin_Station'] = 'Bettembourg'
train_data.loc[train_data['Origin_Station'] == 'Boulou-Perthus-(Le)', 'Origin_Station'] = 'Boulou-Perthus'
train_data.loc[train_data['Origin_Station'] == 'Champigneulles', 'Origin_Station'] = 'Champigneulles'
train_data.loc[train_data['Origin_Station'] == 'Clip Intermodal Sp.zo.o( Poznan)', 'Origin_Station'] = 'Poznan'
train_data.loc[train_data['Origin_Station'] == 'Esch-sur-Alzette', 'Origin_Station'] = 'Esch-sur-Alzette'
train_data.loc[train_data['Origin_Station'] == 'Gent-Zeehaven', 'Origin_Station'] = 'Gent'
train_data.loc[train_data['Origin_Station'] == 'Kiel Schwedenkai KV', 'Origin_Station'] = 'Kiel'
train_data.loc[train_data['Origin_Station'] == 'Lyon-Guillotiere-Port-Herriot', 'Origin_Station'] = 'Lyon'
train_data.loc[train_data['Origin_Station'] == 'Rostock Seehafen', 'Origin_Station'] = 'Rostock'
train_data.loc[train_data['Origin_Station'] == 'Trieste Campo Marzio Rive', 'Origin_Station'] = 'Trieste'
train_data.loc[train_data['Origin_Station'] == 'Valenton-Local', 'Origin_Station'] = 'Valenton'
train_data.loc[train_data['Origin_Station'] == 'Zeebrugge-Ramsk.-Brittania-cont', 'Origin_Station'] = 'Zeebrugge'
train_data.loc[train_data['Origin_Station'] == 'Mertert-Port', 'Origin_Station'] = 'Mertert'

train_data.loc[train_data['Destination_Station'] == 'Antwerpen-Buitenschoor-Combinant', 'Destination_Station'] = 'Antwerp'
train_data.loc[train_data['Destination_Station'] == 'Barcelona can Tunis', 'Destination_Station'] = 'Barcelona'
train_data.loc[train_data['Destination_Station'] == 'Bettembourg-Container-Terminal', 'Destination_Station'] = 'Bettembourg'
train_data.loc[train_data['Destination_Station'] == 'Boulou-Perthus-(Le)', 'Destination_Station'] = 'Boulou-Perthus'
train_data.loc[train_data['Destination_Station'] == 'Champigneulles', 'Destination_Station'] = 'Champigneulles'
train_data.loc[train_data['Destination_Station'] == 'Clip Intermodal Sp.zo.o( Poznan)', 'Destination_Station'] = 'Poznan'
train_data.loc[train_data['Destination_Station'] == 'Esch-sur-Alzette', 'Destination_Station'] = 'Esch-sur-Alzette'
train_data.loc[train_data['Destination_Station'] == 'Gent-Zeehaven', 'Destination_Station'] = 'Gent'
train_data.loc[train_data['Destination_Station'] == 'Kiel Schwedenkai KV', 'Destination_Station'] = 'Kiel'
train_data.loc[train_data['Destination_Station'] == 'Lyon-Guillotiere-Port-Herriot', 'Destination_Station'] = 'Lyon'
train_data.loc[train_data['Destination_Station'] == 'Rostock Seehafen', 'Destination_Station'] = 'Rostock'
train_data.loc[train_data['Destination_Station'] == 'Trieste Campo Marzio Rive', 'Destination_Station'] = 'Trieste'
train_data.loc[train_data['Destination_Station'] == 'Valenton-Local', 'Destination_Station'] = 'Valenton'
train_data.loc[train_data['Destination_Station'] == 'Zeebrugge-Ramsk.-Brittania-cont', 'Destination_Station'] = 'Zeebrugge'

station_data.loc[station_data['Station_Name'] == 'Antwerpen-Buitenschoor-Combinant', 'Station_Name'] = 'Antwerp'
station_data.loc[station_data['Station_Name'] == 'Barcelona can Tunis', 'Station_Name'] = 'Barcelona'
station_data.loc[station_data['Station_Name'] == 'Bettembourg-Container-Terminal', 'Station_Name'] = 'Bettembourg'
station_data.loc[station_data['Station_Name'] == 'Boulou-Perthus-(Le)', 'Station_Name'] = 'Boulou-Perthus'
station_data.loc[station_data['Station_Name'] == 'Champigneulles', 'Station_Name'] = 'Champigneulles'
station_data.loc[station_data['Station_Name'] == 'Clip Intermodal Sp.zo.o( Poznan)', 'Station_Name'] = 'Poznan'
station_data.loc[station_data['Station_Name'] == 'Esch-sur-Alzette', 'Station_Name'] = 'Esch-sur-Alzette'
station_data.loc[station_data['Station_Name'] == 'Gent-Zeehaven', 'Station_Name'] = 'Gent'
station_data.loc[station_data['Station_Name'] == 'Kiel Schwedenkai KV', 'Station_Name'] = 'Kiel'
station_data.loc[station_data['Station_Name'] == 'Lyon-Guillotiere-Port-Herriot', 'Station_Name'] = 'Lyon'
station_data.loc[station_data['Station_Name'] == 'Rostock Seehafen', 'Station_Name'] = 'Rostock'
station_data.loc[station_data['Station_Name'] == 'Trieste Campo Marzio Rive', 'Station_Name'] = 'Trieste'
station_data.loc[station_data['Station_Name'] == 'Valenton-Local', 'Station_Name'] = 'Valenton'
station_data.loc[station_data['Station_Name'] == 'Zeebrugge-Ramsk.-Brittania-cont', 'Station_Name'] = 'Zeebrugge'
station_data.loc[station_data['Station_Name'] == 'Mertert-Port', 'Station_Name'] = 'Mertert'

##Fix any Datatypes

In [33]:
train_data['Planned_Departure'] = pd.to_datetime(train_data['Planned_Departure'],infer_datetime_format=True)
train_data['Actual_Departure'] = pd.to_datetime(train_data['Actual_Departure'],infer_datetime_format=True)
train_data['Planned_Arrival'] = pd.to_datetime(train_data['Planned_Arrival'],infer_datetime_format=True)
train_data['Actual_Arrival'] = pd.to_datetime(train_data['Actual_Arrival'],infer_datetime_format=True)

train_position['DateH_position'] = pd.to_datetime(train_position['DateH_position'],infer_datetime_format=True)

station_data['Planned_Departure'] = pd.to_datetime(station_data['Planned_Departure'],infer_datetime_format=True)
station_data['Actual_Departure'] = pd.to_datetime(station_data['Actual_Departure'],infer_datetime_format=True)
station_data['Planned_Arrival'] = pd.to_datetime(station_data['Planned_Arrival'],infer_datetime_format=True)
station_data['Actual_Arrival'] = pd.to_datetime(station_data['Actual_Arrival'],infer_datetime_format=True)

In [34]:
#recode values
train_data['Cancellation_Reason'] = train_data['Cancellation_Reason'].replace({'AUTRE_OPERATEUR':'Other Operator','AUTRE OPÉRATEUR':'Other Operator','FERIE':'Holiday','GREVE':'Labor Action','FORCE_MAJEURE':'Act of God'})

In [35]:
station_data

,IDTRAIN,IDGARE,IDTRAIN_JALON,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Planned_Arrival,Actual_Arrival,Planned_Departure,Actual_Departure
0,255,80248,3507,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,2019-03-04 01:50:00,NaT
1,255,85792,3510,2,Bettembourg-frontière,LU,49.467859,6.109153,2019-03-04 04:50:00,NaT,2019-03-04 04:50:00,NaT
2,255,87910,3509,3,Hagondange,FR,49.245710,6.162933,2019-03-04 05:30:00,NaT,2019-03-04 05:30:00,NaT
3,255,87942,3511,4,Woippy,FR,49.149512,6.156061,2019-03-04 05:40:00,NaT,2019-03-04 05:40:00,NaT
4,255,87939,3512,5,Metz-Chambiere,FR,49.126336,6.159898,2019-03-04 05:55:00,NaT,2019-03-04 05:55:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
72526,22417,67920,120915,7,Ehrang,DE,49.802000,6.686000,2021-04-10 08:43:00,NaT,2021-04-10 09:14:00,NaT
72527,22417,85881,120918,8,Apach-Frontiere,FR,49.459076,6.372680,2021-04-10 11:49:00,NaT,2021-04-10 11:50:00,NaT
72528,22417,87935,120919,9,Metz-Sablon,FR,49.096407,6.166320,2021-04-10 12:48:00,NaT,2021-04-10 12:55:00,NaT
72529,22417,85791,120916,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,2021-04-10 13:54:00,NaT,2021-04-10 13:55:00,NaT


In [36]:
#create individual rows for each timestamp
station_data_stops = station_data.melt(id_vars=['IDTRAIN','IDGARE','Station_Order','Station_Name','Station_Country','Station_Latitude','Station_Longitude','Actual_Arrival','Actual_Departure'], value_vars=['Planned_Arrival','Planned_Departure'], var_name='Schedule', value_name='Plan_Timestamp')
station_data_stops = station_data_stops.sort_values(by=['IDTRAIN','Station_Order'])
station_data_stops = station_data_stops.reset_index(drop=True)

#Remove first and last rows in each train (unnecessary rows)
station_data_stops = station_data_stops.drop(station_data_stops.groupby('IDTRAIN').head(1).index)
station_data_stops = station_data_stops.drop(station_data_stops.groupby('IDTRAIN').tail(1).index)
station_data_stops = station_data_stops.reset_index(drop=True)

#enumerate time events
station_data_stops['Timestamp_Order'] = station_data_stops.groupby(['IDTRAIN']).cumcount()

#calculate event O|D Pairs
origin = station_data_stops[['Station_Name']]
origin = origin.rename(columns={'Station_Name':'Origin'})
origin = origin.shift(periods=1)
origin = origin.drop(origin.head(1).index)
destination = station_data_stops[['Station_Name']]
destination = destination.rename(columns={'Station_Name':'Destination'})
od = origin.join(destination)
od['OD_Pair_Station'] = od['Origin'].astype(str) + '|' + od['Destination'].astype(str)
od.loc[od['Origin'] == od['Destination'], 'OD_Pair_Type'] = 'Waypoint'
od.loc[od['Origin'] != od['Destination'], 'OD_Pair_Type'] = 'Transit'
od = od[['OD_Pair_Station','OD_Pair_Type']]
station_data_stops = station_data_stops.join(od)

station_data_stops.loc[(station_data_stops['Timestamp_Order'] == 0) & (station_data_stops['Station_Name'] == 'Bettembourg'), 'OD_Pair_Station'] = 'Bettembourg Origin'
station_data_stops.loc[(station_data_stops['Timestamp_Order'] == 0) & (station_data_stops['Station_Name'] != 'Bettembourg'), 'OD_Pair_Station'] = 'Other Origin'
station_data_stops.loc[station_data_stops['Timestamp_Order'] == 0, 'OD_Pair_Type'] = 'Origin'

#Calculate Departure and Arrival Variances for each stop
station_data_stops.loc[station_data_stops['Schedule'] == 'Planned_Arrival', 'Arrive_Variance'] = station_data_stops['Actual_Arrival'] - station_data_stops['Plan_Timestamp']
station_data_stops.loc[station_data_stops['Schedule'] == 'Planned_Departure', 'Depart_Variance'] = station_data_stops['Actual_Departure'] - station_data_stops['Plan_Timestamp']

#calculate planned travel and waiting times for each stop
station_data_stops['Time_From_Prior_Planned'] = station_data_stops.groupby(['IDTRAIN'])['Plan_Timestamp'].diff()
station_data_stops['Time_From_Prior_Planned'] = station_data_stops.groupby(['IDTRAIN'])['Time_From_Prior_Planned'].fillna(value=pd.Timedelta(seconds=0), limit=1)

#Combine Actuals Timestamps to one column
station_data_stops['Actual_Timestamp'] = station_data_stops['Actual_Arrival'].fillna(station_data_stops['Actual_Departure'])

#calculate actual travel and waiting times for each stop
station_data_stops['Time_From_Prior_Actual'] = station_data_stops.groupby(['IDTRAIN'])['Actual_Timestamp'].diff()
station_data_stops['Time_From_Prior_Actual'] = station_data_stops.groupby(['IDTRAIN'])['Time_From_Prior_Actual'].fillna(value=pd.Timedelta(seconds=0), limit=1)

#convert timedeltas to minutes
depart_variance_mins = []
for i in station_data_stops['Depart_Variance']:
    depart_variance_mins.append(i.total_seconds()/60)
station_data_stops['Depart_Variance_Mins'] = depart_variance_mins

arrive_variance_mins = []
for i in station_data_stops['Arrive_Variance']:
    arrive_variance_mins.append(i.total_seconds()/60)
station_data_stops['Arrive_Variance_Mins'] = arrive_variance_mins

time_from_prior_mins = []
for i in station_data_stops['Time_From_Prior_Planned']:
    time_from_prior_mins.append(i.total_seconds()/60)
station_data_stops['Time_From_Prior_Plan_Mins'] = time_from_prior_mins
station_data_stops.loc[station_data_stops['Timestamp_Order'] == 0, 'Time_From_Prior_Plan_Mins'] = np.nan

time_from_prior_mins = []
for i in station_data_stops['Time_From_Prior_Actual']:
    time_from_prior_mins.append(i.total_seconds()/60)
station_data_stops['Time_From_Prior_Actual_Mins'] = time_from_prior_mins
station_data_stops.loc[station_data_stops['Timestamp_Order'] == 0, 'Time_From_Prior_Actual_Mins'] = np.nan

#Columns for travel time and idle time
station_data_stops.loc[station_data_stops['Schedule'] == 'Planned_Arrival', 'Travel_Time_Mins'] = station_data_stops['Time_From_Prior_Plan_Mins']
station_data_stops.loc[station_data_stops['Schedule'] == 'Planned_Departure', 'Idle_Time_Mins'] = station_data_stops['Time_From_Prior_Plan_Mins']
station_data_stops.loc[station_data_stops['Timestamp_Order'] == 0, 'Time_From_Prior_Plan_Mins'] = 0

#Calculate Distance Between Events
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

station_data_stops['KM_Distance_Event'] = \
    np.concatenate(station_data_stops.groupby('IDTRAIN')
                     .apply(lambda x: haversine(x['Station_Latitude'], x['Station_Longitude'],
                                                x['Station_Latitude'].shift(), x['Station_Longitude'].shift())).values)

station_data_stops['KM_Distance_Event'] = station_data_stops.groupby(['IDTRAIN'])['KM_Distance_Event'].fillna(value=0, limit=1)

#calculate cummulative distance
station_data_stops['Cumm_Distance_KM'] = station_data_stops.groupby(['IDTRAIN'])['KM_Distance_Event'].cumsum()

#Calculate Event Speed
station_data_stops['KM_HR_Event'] = (station_data_stops['KM_Distance_Event']/(station_data_stops['Time_From_Prior_Plan_Mins']/60)).fillna(0)

#Calculate percentage of actual timestamps per journey
train_data['Cancelled_Train_Bin'] = [0 if x else 1 for x in train_data['Cancellation_Reason'].isna()]
cancelled_list = train_data[['IDTRAIN','Cancelled_Train_Bin']]
station_data_stops = pd.merge(station_data_stops, cancelled_list, how='left', left_on=['IDTRAIN'], right_on=['IDTRAIN'])
station_data_stops['Actual_Timestamp_Valid'] = [0 if x else 1 for x in station_data_stops['Actual_Timestamp'].isna()]
station_data_stops.loc[station_data_stops['Cancelled_Train_Bin'] == 1,'Actual_Timestamp_Valid'] = np.nan
station_data_actuals = station_data_stops.groupby(['IDTRAIN'],as_index=False).agg(Actual_Timestamp_Perc=pd.NamedAgg(column='Actual_Timestamp_Valid', aggfunc='mean'))
station_data_stops = pd.merge(station_data_stops,station_data_actuals,left_on=['IDTRAIN'],right_on=['IDTRAIN'],how='left')
actual_perc = station_data_actuals[['IDTRAIN','Actual_Timestamp_Perc']]
train_data = pd.merge(train_data, actual_perc ,left_on=['IDTRAIN'],right_on=['IDTRAIN'],how='left')

#Apply Train Departure delay actual timestamps
station_data_stops.loc[station_data_stops.groupby('IDTRAIN',as_index=False).head(1).index,'Time_From_Prior_Actual_Mins'] = np.nan
station_data_stops['Time_From_Prior_Actual_Mins'] = station_data_stops['Time_From_Prior_Actual_Mins'].fillna(station_data_stops['Depart_Variance_Mins'])

#station_data_stops['Time_From_Prior_Actual_Mins'] = station_data_stops.groupby(['IDTRAIN'])['Time_From_Prior_Actual_Mins'].apply(lambda x: x.fillna(value=init_delay, limit=1))
#station_data_stops.loc[station_data_stops['Timestamp_Order'] == 0, 'Time_From_Prior_Actual_Mins'] = station_data_stops.groupby(['IDTRAIN'])['Depart_Variance_Mins'].first()
    
#calculate cummulative minutes
station_data_stops['Cumm_Schedule_Mins'] = station_data_stops.groupby(['IDTRAIN'])['Time_From_Prior_Plan_Mins'].cumsum()
station_data_stops['Cumm_Actual_Mins'] = station_data_stops.groupby(['IDTRAIN'])['Time_From_Prior_Actual_Mins'].cumsum()

station_data_stops

,IDTRAIN,IDGARE,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Actual_Arrival,Actual_Departure,Schedule,...,Travel_Time_Mins,Idle_Time_Mins,KM_Distance_Event,Cumm_Distance_KM,KM_HR_Event,Cancelled_Train_Bin,Actual_Timestamp_Valid,Actual_Timestamp_Perc,Cumm_Schedule_Mins,Cumm_Actual_Mins
0,255,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,Planned_Departure,...,NaN,NaN,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,NaN
1,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,180.0,NaN,2.894467,2.894467,0.964822,0,0.0,0.0,180.0,NaN
2,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Departure,...,NaN,0.0,0.000000,2.894467,0.000000,0,0.0,0.0,180.0,NaN
3,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Arrival,...,40.0,NaN,25.007052,27.901519,37.510578,0,0.0,0.0,220.0,NaN
4,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Departure,...,NaN,0.0,0.000000,27.901519,0.000000,0,0.0,0.0,220.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116627,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Arrival,...,58.0,NaN,43.015834,662.661147,44.499138,0,0.0,0.0,1058.0,NaN
116628,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Departure,...,NaN,7.0,0.000000,662.661147,0.000000,0,0.0,0.0,1065.0,NaN
116629,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,59.0,NaN,41.511206,704.172353,42.214786,0,0.0,0.0,1124.0,NaN
116630,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Departure,...,NaN,1.0,0.000000,704.172353,0.000000,0,0.0,0.0,1125.0,NaN


##Aggregate Features for Station Data

In [37]:
#Calculate Origin and Destination Stations, total travel distance in km, and total, travel, idle times for train in mins
station_data_agg = station_data_stops.groupby(['IDTRAIN'],as_index=False).agg(Origin_Station=pd.NamedAgg(column='Station_Name', aggfunc='first')
  ,Destination_Station=pd.NamedAgg(column='Station_Name', aggfunc='last')
  ,Train_Distance_KM=pd.NamedAgg(column='KM_Distance_Event', aggfunc='sum')
  ,Train_Total_Time_Mins=pd.NamedAgg(column='Time_From_Prior_Plan_Mins', aggfunc='sum')
  ,Train_Travel_Time_Mins=pd.NamedAgg(column='Travel_Time_Mins', aggfunc='sum')
  ,Train_Idle_Time_Mins=pd.NamedAgg(column='Idle_Time_Mins', aggfunc='sum')
  )

#Calculate Train Journey Time Variance
station_data_agg['Train_Depart_Variance_Mins'] = station_data_stops.groupby(['IDTRAIN'])['Depart_Variance_Mins'].first()
station_data_agg['Train_Arrive_Variance_Mins'] = station_data_stops.groupby(['IDTRAIN'])['Arrive_Variance_Mins'].last()

#Calculate travel speed and overall speed
station_data_agg['Travel_KM_HR'] = (station_data_agg['Train_Distance_KM']/(station_data_agg['Train_Travel_Time_Mins']/60)).fillna(0)
station_data_agg['Total_KM_HR'] = (station_data_agg['Train_Distance_KM']/(station_data_agg['Train_Total_Time_Mins']/60)).fillna(0)

station_data = pd.merge(station_data_stops, station_data_agg, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='left')

station_data

,IDTRAIN,IDGARE,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Actual_Arrival,Actual_Departure,Schedule,...,Origin_Station,Destination_Station,Train_Distance_KM,Train_Total_Time_Mins,Train_Travel_Time_Mins,Train_Idle_Time_Mins,Train_Depart_Variance_Mins,Train_Arrive_Variance_Mins,Travel_KM_HR,Total_KM_HR
0,255,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,Planned_Departure,...,Bettembourg,Champigneulles,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886
1,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,Bettembourg,Champigneulles,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886
2,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Departure,...,Bettembourg,Champigneulles,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886
3,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Arrival,...,Bettembourg,Champigneulles,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886
4,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Departure,...,Bettembourg,Champigneulles,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116627,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Arrival,...,Kiel,Bettembourg,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309
116628,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Departure,...,Kiel,Bettembourg,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309
116629,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,Kiel,Bettembourg,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309
116630,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Departure,...,Kiel,Bettembourg,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309


## Calculate features across aggregations

In [38]:
station_data['Stage_Share_Journey_Distance'] = station_data['KM_Distance_Event']/station_data['Train_Distance_KM']
station_data['Stage_Share_Journey_Time_Mins'] = station_data['Time_From_Prior_Plan_Mins']/station_data['Train_Total_Time_Mins']

station_data

,IDTRAIN,IDGARE,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Actual_Arrival,Actual_Departure,Schedule,...,Train_Distance_KM,Train_Total_Time_Mins,Train_Travel_Time_Mins,Train_Idle_Time_Mins,Train_Depart_Variance_Mins,Train_Arrive_Variance_Mins,Travel_KM_HR,Total_KM_HR,Stage_Share_Journey_Distance,Stage_Share_Journey_Time_Mins
0,255,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,Planned_Departure,...,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886,0.000000,0.000000
1,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886,0.005326,0.649819
2,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Departure,...,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886,0.000000,0.000000
3,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Arrival,...,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886,0.046011,0.144404
4,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Departure,...,543.496559,277.0,277.0,0.0,NaN,NaN,117.724886,117.724886,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116627,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Arrival,...,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309,0.060837,0.051464
116628,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Departure,...,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309,0.000000,0.006211
116629,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309,0.058709,0.052351
116630,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Departure,...,707.066820,1127.0,991.0,136.0,6.0,-54.0,42.809293,37.643309,0.000000,0.000887


# Generate Features

## Location Groupings

City and Country

In [39]:
train_data['Origin_City_Country'] = train_data['Origin_Station'] + ', ' + train_data['Origin_Country']
train_data['Destination_City_Country'] = train_data['Destination_Station'] + ', ' + train_data['Destination_Country']
station_data['Station_City_Country'] = station_data['Station_Name'] + ', ' + station_data['Station_Country']

OD Pairs

In [40]:
train_data['OD_Pair'] = train_data['Origin_Station'] + ' | ' + train_data['Destination_Station']

In [41]:
station_data['OD_Pair'] = station_data['Origin_Station'] + ' | ' + station_data['Destination_Station']

ARCs

In [42]:
train_data.loc[train_data['OD_Pair'] == 'Champigneulles | Bettembourg', 'Arc'] = 'Champigneulles'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Champigneulles', 'Arc'] = 'Champigneulles'
train_data.loc[train_data['OD_Pair'] == 'Trieste | Bettembourg', 'Arc'] = 'Trieste'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Trieste', 'Arc'] = 'Trieste'
train_data.loc[train_data['OD_Pair'] == 'Zeebrugge | Bettembourg', 'Arc'] = 'Zeebrugge'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Zeebrugge', 'Arc'] = 'Zeebrugge'
train_data.loc[train_data['OD_Pair'] == 'Lyon | Bettembourg', 'Arc'] = 'Lyon'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Lyon', 'Arc'] = 'Lyon'
train_data.loc[train_data['OD_Pair'] == 'Antwerp | Bettembourg', 'Arc'] = 'Antwerp'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Antwerp', 'Arc'] = 'Antwerp'
train_data.loc[train_data['OD_Pair'] == 'Boulou-Perthus | Bettembourg', 'Arc'] = 'Boulou-Perthus'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Boulou-Perthus', 'Arc'] = 'Boulou-Perthus'
train_data.loc[train_data['OD_Pair'] == 'Gent | Bettembourg', 'Arc'] = 'Gent'
train_data.loc[train_data['OD_Pair'] == 'Valenton | Esch-sur-Alzette', 'Arc'] = 'Valenton'
train_data.loc[train_data['OD_Pair'] == 'Esch-sur-Alzette | Valenton', 'Arc'] = 'Valenton'
train_data.loc[train_data['OD_Pair'] == 'Kiel | Bettembourg', 'Arc'] = 'Kiel'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Kiel', 'Arc'] = 'Kiel'
train_data.loc[train_data['OD_Pair'] == 'Poznan | Bettembourg', 'Arc'] = 'Poznan'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Poznan', 'Arc'] = 'Poznan'
train_data.loc[train_data['OD_Pair'] == 'Barcelona | Bettembourg', 'Arc'] = 'Barcelona'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Barcelona', 'Arc'] = 'Barcelona'
train_data.loc[train_data['OD_Pair'] == 'Rostock | Bettembourg', 'Arc'] = 'Rostock'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Rostock', 'Arc'] = 'Rostock'
train_data.loc[train_data['OD_Pair'] == 'Mertert | Bettembourg', 'Arc'] = 'Mertert'
train_data.loc[train_data['OD_Pair'] == 'Bettembourg | Mertert', 'Arc'] = 'Mertert'


station_data.loc[station_data['OD_Pair'] == 'Champigneulles | Bettembourg', 'Arc'] = 'Champigneulles'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Champigneulles', 'Arc'] = 'Champigneulles'
station_data.loc[station_data['OD_Pair'] == 'Trieste | Bettembourg', 'Arc'] = 'Trieste'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Trieste', 'Arc'] = 'Trieste'
station_data.loc[station_data['OD_Pair'] == 'Zeebrugge | Bettembourg', 'Arc'] = 'Zeebrugge'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Zeebrugge', 'Arc'] = 'Zeebrugge'
station_data.loc[station_data['OD_Pair'] == 'Lyon | Bettembourg', 'Arc'] = 'Lyon'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Lyon', 'Arc'] = 'Lyon'
station_data.loc[station_data['OD_Pair'] == 'Antwerp | Bettembourg', 'Arc'] = 'Antwerp'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Antwerp', 'Arc'] = 'Antwerp'
station_data.loc[station_data['OD_Pair'] == 'Boulou-Perthus | Bettembourg', 'Arc'] = 'Boulou-Perthus'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Boulou-Perthus', 'Arc'] = 'Boulou-Perthus'
station_data.loc[station_data['OD_Pair'] == 'Gent | Bettembourg', 'Arc'] = 'Gent'
station_data.loc[station_data['OD_Pair'] == 'Valenton | Esch-sur-Alzette', 'Arc'] = 'Valenton'
station_data.loc[station_data['OD_Pair'] == 'Esch-sur-Alzette | Valenton', 'Arc'] = 'Valenton'
station_data.loc[station_data['OD_Pair'] == 'Kiel | Bettembourg', 'Arc'] = 'Kiel'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Kiel', 'Arc'] = 'Kiel'
station_data.loc[station_data['OD_Pair'] == 'Poznan | Bettembourg', 'Arc'] = 'Poznan'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Poznan', 'Arc'] = 'Poznan'
station_data.loc[station_data['OD_Pair'] == 'Barcelona | Bettembourg', 'Arc'] = 'Barcelona'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Barcelona', 'Arc'] = 'Barcelona'
station_data.loc[station_data['OD_Pair'] == 'Rostock | Bettembourg', 'Arc'] = 'Rostock'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Rostock', 'Arc'] = 'Rostock'
station_data.loc[station_data['OD_Pair'] == 'Mertert | Bettembourg', 'Arc'] = 'Mertert'
station_data.loc[station_data['OD_Pair'] == 'Bettembourg | Mertert', 'Arc'] = 'Mertert'

## Time Variances

Arrival and Departure Variance as datetime.delta

In [43]:
train_data['Depart_Variance'] = train_data['Actual_Departure'] - train_data['Planned_Departure'] #departure delay
train_data['Arrive_Variance'] = train_data['Actual_Arrival'] - train_data['Planned_Arrival'] #arrival delay

Departure and Arrival Variance as mins

In [44]:
depart_variance_mins = []
for i in train_data['Depart_Variance']:
    depart_variance_mins.append(i.total_seconds()/60)
train_data['Depart_Variance_Mins'] = depart_variance_mins

train_data['Depart_Variance_Mins_Pos'] = [x if x >= 0 else 0 if x < 0 else x for x in train_data['Depart_Variance_Mins']]
station_data['Depart_Variance_Mins_Pos'] = [x if x >= 0 else 0 if x < 0 else x for x in station_data['Depart_Variance_Mins']]

arrive_variance_mins = []
for i in train_data['Arrive_Variance']:
    arrive_variance_mins.append(i.total_seconds()/60)
train_data['Arrive_Variance_Mins'] = arrive_variance_mins

train_data['Arrive_Variance_Mins_Pos'] = [x if x >= 0 else 0 if x < 0 else x for x in train_data['Arrive_Variance_Mins']]
station_data['Arrive_Variance_Mins_Pos'] = [x if x >= 0 else 0 if x < 0 else x for x in station_data['Arrive_Variance_Mins']]

Late Departure Binary Recode

In [45]:
# for any delay
train_data['Late_Departure_Bin'] = [1 if x > 0 else 0 if x <= 0 else x for x in train_data['Depart_Variance_Mins']]
train_data['Late_Arrival_Bin'] = [1 if x > 0 else 0 if x <= 0 else x for x in train_data['Arrive_Variance_Mins']]

station_data['Late_Departure_Bin'] = [1 if x > 0 else 0 if x <= 0 else x for x in station_data['Depart_Variance_Mins']]
station_data['Late_Arrival_Bin'] = [1 if x > 0 else 0 if x <= 0 else x for x in station_data['Arrive_Variance_Mins']]

# for 15 min delay
train_data['Late_Departure_Bin_15'] = [1 if x >= 15 else 0 if x < 15 else x for x in train_data['Depart_Variance_Mins']]
train_data['Late_Arrival_Bin_15'] = [1 if x >= 15 else 0 if x < 15 else x for x in train_data['Arrive_Variance_Mins']]

station_data['Late_Departure_Bin_15'] = [1 if x >= 15 else 0 if x < 15 else x for x in station_data['Depart_Variance_Mins']]
station_data['Late_Arrival_Bin_15'] = [1 if x >= 15 else 0 if x < 15 else x for x in station_data['Arrive_Variance_Mins']]

# for 30 min delay
train_data['Late_Departure_Bin_30'] = [1 if x >= 30 else 0 if x < 30 else x for x in train_data['Depart_Variance_Mins']]
train_data['Late_Arrival_Bin_30'] = [1 if x >= 30 else 0 if x < 30 else x for x in train_data['Arrive_Variance_Mins']]

station_data['Late_Departure_Bin_30'] = [1 if x >= 30 else 0 if x < 30 else x for x in station_data['Depart_Variance_Mins']]
station_data['Late_Arrival_Bin_30'] = [1 if x >= 30 else 0 if x < 30 else x for x in station_data['Arrive_Variance_Mins']]

# for 60 min delay
train_data['Late_Departure_Bin_60'] = [1 if x >= 60 else 0 if x < 60 else x for x in train_data['Depart_Variance_Mins']]
train_data['Late_Arrival_Bin_60'] = [1 if x >= 60 else 0 if x < 60 else x for x in train_data['Arrive_Variance_Mins']]

station_data['Late_Departure_Bin_60'] = [1 if x >= 60 else 0 if x < 60 else x for x in station_data['Depart_Variance_Mins']]
station_data['Late_Arrival_Bin_60'] = [1 if x >= 60 else 0 if x < 60 else x for x in station_data['Arrive_Variance_Mins']]

# for 180 min delay
train_data['Late_Departure_Bin_180'] = [1 if x >= 180 else 0 if x < 180 else x for x in train_data['Depart_Variance_Mins']]
train_data['Late_Arrival_Bin_180'] = [1 if x >= 180 else 0 if x < 180 else x for x in train_data['Arrive_Variance_Mins']]

station_data['Late_Departure_Bin_180'] = [1 if x >= 180 else 0 if x < 180 else x for x in station_data['Depart_Variance_Mins']]
station_data['Late_Arrival_Bin_180'] = [1 if x >= 180 else 0 if x < 180 else x for x in station_data['Arrive_Variance_Mins']]

#Binary for On-Time Trains
station_data['On Time Bin'] = [0 if x >= 60 else 1 for x in station_data['Depart_Variance_Mins']]
station_data['On Time Bin'] = [0 if x >= 60 else 1 for x in station_data['Arrive_Variance_Mins']]
station_data.loc[station_data['Cancelled_Train_Bin'] == 1, 'On Time Bin'] = np.nan

train_data['On Time Bin'] = [0 if x >= 60 else 1 for x in train_data['Depart_Variance_Mins']]
train_data['On Time Bin'] = [0 if x >= 60 else 1 for x in train_data['Arrive_Variance_Mins']]
train_data.loc[train_data['Cancelled_Train_Bin'] == 1, 'On Time Bin'] = np.nan

Recoded Depart Variance into Bins

In [46]:
#bins = list(range((outlier_lower_threshold_hours*-1*60),(outlier_upper_threshold_hours*60),departure_bin_mins))
#bin_labels = []
#for i in bins:
#  bin_labels.append(str(i) + ' - ' + str(i+departure_bin_mins)) 
#bins.append(outlier_upper_threshold_hours*60+departure_bin_mins)
#train_data['Depart_Variance_Mins_Cat'] = pd.cut(train_data['Depart_Variance_Mins'], bins=bins, labels=bin_labels)

## Calendar Features


Scheduled Date

In [47]:
train_data['Planned_Departure_Date'] = pd.to_datetime(train_data['Planned_Departure']).dt.date
train_data['Planned_Arrival_Date'] = pd.to_datetime(train_data['Planned_Arrival']).dt.date

station_data['Planned_Departure_Date'] = station_data_stops.groupby(['IDTRAIN'])['Plan_Timestamp'].first()
station_data['Planned_Departure_Date'] = pd.to_datetime(station_data['Planned_Departure_Date']).dt.date

station_data['Planned_Arrival_Date'] = station_data_stops.groupby(['IDTRAIN'])['Plan_Timestamp'].last()
station_data['Planned_Arrival_Date'] = pd.to_datetime(station_data['Planned_Arrival_Date']).dt.date

Designate DOW

In [48]:
weekdays = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}

train_data['Planned_Departure_DOW'] = pd.to_datetime(train_data['Planned_Departure_Date']).dt.dayofweek.map(weekdays)
train_data['Planned_Arrival_DOW'] = pd.to_datetime(train_data['Planned_Arrival_Date']).dt.dayofweek.map(weekdays)

station_data['Planned_Departure_DOW'] = pd.to_datetime(station_data['Planned_Departure_Date']).dt.dayofweek.map(weekdays)
station_data['Planned_Arrival_DOW'] = pd.to_datetime(station_data['Planned_Arrival_Date']).dt.dayofweek.map(weekdays)

Week of Year

In [49]:
train_data['Depart_Week_Num'] = pd.to_datetime(train_data['Planned_Departure_Date']).dt.isocalendar().week

## Binary Fields for Filtering

Binary Fields for Avail/Missing Time Data

In [50]:
train_data["Actual_Departure_Bin"] = [0 if x else 1 for x in train_data["Actual_Departure"].isna()]
train_data.loc[train_data['Cancelled_Train_Bin'] == 1, 'Actual_Departure_Bin'] = np.nan
station_data["Actual_Departure_Bin"] = [0 if x else 1 for x in station_data["Actual_Departure"].isna()]
station_data.loc[station_data['Cancelled_Train_Bin'] == 1, 'Actual_Departure_Bin'] = np.nan

train_data["Actual_Arrival_Bin"] = [0 if x else 1 for x in train_data["Actual_Arrival"].isna()]
train_data.loc[train_data['Cancelled_Train_Bin'] == 1, 'Actual_Arrival_Bin'] = np.nan
station_data["Actual_Arrival_Bin"] = [0 if x else 1 for x in station_data["Actual_Arrival"].isna()]
station_data.loc[station_data['Cancelled_Train_Bin'] == 1, 'Actual_Arrival_Bin'] = np.nan

Binary Fields for Arrival and Departure Trains

In [51]:
#Binary to define whether it is the origin or destination
station_data['Origin_Station_Bin'] = [1 if x == 1 else 0 for x in station_data['Station_Order']] 
station_data['Destination_Station_Bin'] = [1 if x == 9999 else 0 for x in station_data['Station_Order']] 

Binary for Bettembourg Departure

In [52]:
train_data['BB_Departure_Bin'] = [1 if x == 'Bettembourg' else 0 for x in train_data['Origin_Station']]

Binary for Cancelled Train

In [53]:
#train_data['Cancelled_Train_Bin'] = [0 if x else 1 for x in train_data['Cancellation_Reason'].isna()]

In [54]:
#Binary for full intermediate timestamps
train_data['Full_Journey_Timestamp_Bin'] = [1 if x == 1 else 0 for x in train_data['Actual_Timestamp_Perc']]
train_data.loc[train_data['Cancelled_Train_Bin'] == 1, 'Full_Journey_Timestamp_Bin'] = np.nan

In [55]:
#add bin fields to station data
station_data = pd.merge(station_data, train_data[['IDTRAIN','BB_Departure_Bin','Full_Journey_Timestamp_Bin']],how='left',left_on=['IDTRAIN'], right_on=['IDTRAIN'])

## Aggregations for "Long" Files

Wagons per Train

In [56]:
#Calculate the number of wagons per train
train_wagon_count = wagon_data.groupby(['IDTRAIN_LOT'],as_index=False).agg(wagon_count=pd.NamedAgg(column='Wagon_Order', aggfunc=max))

Train duration from train_position lat/long - WIP as Data quality poor

In [57]:
#train level departure delay
initial_delay = train_data[['IDTRAIN','Depart_Variance_Mins']]

In [58]:
train_position = train_position.rename(columns={'DateH_position': 'Timestamp'})

#enumerate events
train_position = train_position.sort_values(by=['IDTRAIN','Timestamp'])
train_position = train_position.reset_index(drop=True)
train_position['Event_Order'] = train_position.groupby('IDTRAIN').cumcount()

#calculate distance between events
train_position['KM_Distance_Event'] = \
    np.concatenate(train_position.groupby('IDTRAIN')
                     .apply(lambda x: haversine(x['Latitude'], x['Longitude'],
                                                x['Latitude'].shift(), x['Longitude'].shift())).values)

train_position['KM_Distance_Event'] = train_position.groupby(['IDTRAIN'])['KM_Distance_Event'].fillna(value=0, limit=1)

#calculate cummulative distance
train_position['Cumm_Distance_KM'] = train_position.groupby(['IDTRAIN'])['KM_Distance_Event'].cumsum()

#calculate planned travel and waiting times for each stop
train_position['Time_From_Prior_Event'] = train_position.groupby(['IDTRAIN'])['Timestamp'].diff()
train_position['Time_From_Prior_Event'] = train_position.groupby(['IDTRAIN'])['Time_From_Prior_Event'].fillna(value=pd.Timedelta(seconds=0), limit=1)

#convert time deltas to minutes
time_from_prior_mins = []
for i in train_position['Time_From_Prior_Event']:
    time_from_prior_mins.append(i.total_seconds()/60)
train_position['Time_From_Prior_Event_Mins'] = time_from_prior_mins


train_position = pd.merge(train_position, initial_delay, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='left')
train_position.loc[train_position.groupby('IDTRAIN',as_index=False).head(1).index,'Time_From_Prior_Event_Mins'] = np.nan
train_position['Time_From_Prior_Event_Mins'] = train_position['Time_From_Prior_Event_Mins'].fillna(train_position['Depart_Variance_Mins'])

#duration
train_position['Cumm_Mins'] = train_position.groupby(['IDTRAIN'])['Time_From_Prior_Event_Mins'].cumsum()

train_position

,IDTRAIN_POSITION,IDTRAIN,Timestamp,Latitude,Longitude,Event_Order,KM_Distance_Event,Cumm_Distance_KM,Time_From_Prior_Event,Time_From_Prior_Event_Mins,Depart_Variance_Mins,Cumm_Mins
0,98,14688,2020-11-24 09:47:20,49.796185,6.685079,0,0.000000,0.000000,0 days 00:00:00,68.000000,68.0,68.000000
1,99,14688,2020-11-24 11:49:37,49.785930,6.682618,1,1.153908,1.153908,0 days 02:02:17,122.283333,68.0,190.283333
2,100,14688,2020-11-24 12:06:25,49.707500,6.549719,2,12.932013,14.085921,0 days 00:16:48,16.800000,68.0,207.083333
3,101,14688,2020-11-24 12:11:27,49.713070,6.515863,3,2.511952,16.597873,0 days 00:05:02,5.033333,68.0,212.116667
4,102,14688,2020-11-24 12:37:53,49.701965,6.400790,4,8.366366,24.964239,0 days 00:26:26,26.433333,68.0,238.550000
...,...,...,...,...,...,...,...,...,...,...,...,...
25338,24640,22391,2021-04-05 07:27:37,45.904730,13.582446,70,17.223370,1072.369664,0 days 00:14:18,14.300000,70.0,1660.200000
25339,24643,22391,2021-04-05 07:43:41,45.793373,13.594290,71,12.416268,1084.785933,0 days 00:16:04,16.066667,70.0,1676.266667
25340,24646,22391,2021-04-05 07:57:48,45.685750,13.749722,72,16.991531,1101.777463,0 days 00:14:07,14.116667,70.0,1690.383333
25341,24648,22391,2021-04-05 08:10:45,45.639110,13.762757,73,5.284136,1107.061599,0 days 00:12:57,12.950000,70.0,1703.333333


# Join Features

In [59]:
station_data_merge = station_data_agg[['IDTRAIN', 'Train_Distance_KM', 'Train_Total_Time_Mins', 'Train_Travel_Time_Mins', 'Train_Idle_Time_Mins', 'Travel_KM_HR', 'Total_KM_HR']]

train_data = pd.merge(train_data, train_wagon_count, left_on=['IDTRAIN_LOT'], right_on=['IDTRAIN_LOT'], how='left')
train_data = pd.merge(train_data, station_data_merge, left_on=['IDTRAIN'], right_on=['IDTRAIN'], how='left')

train_data

,IDTRAIN,IDTRAIN_LOT,IDTRAIN_JALON_origin,IDGARE_origin,Code_origin,IDGARE_Parent_origin,IDTRAIN_JALON_destination,IDGARE_destination,Code_destination,IDGARE_Parent_destination,...,Actual_Arrival_Bin,BB_Departure_Bin,Full_Journey_Timestamp_Bin,wagon_count,Train_Distance_KM,Train_Total_Time_Mins,Train_Travel_Time_Mins,Train_Idle_Time_Mins,Travel_KM_HR,Total_KM_HR
0,269,269,3619,84831,14108,0.0,3620,80248,30100,25174.0,...,NaN,0,NaN,NaN,543.496559,177.0,177.0,0.0,184.236122,184.236122
1,259,259,3539,80248,30100,25174.0,3540,84831,14108,0.0,...,NaN,1,NaN,NaN,543.496559,114.0,114.0,0.0,286.050820,286.050820
2,362,362,4363,80248,30100,25174.0,4370,85346,3471,0.0,...,NaN,1,NaN,NaN,931.394206,1798.0,1618.0,180.0,34.538722,31.081008
3,274,274,3659,84831,14108,0.0,3660,80248,30100,25174.0,...,NaN,0,NaN,NaN,543.496559,177.0,177.0,0.0,184.236122,184.236122
4,264,264,3579,80248,30100,25174.0,3580,84831,14108,0.0,...,NaN,1,NaN,NaN,543.496559,114.0,114.0,0.0,286.050820,286.050820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14210,20739,22004,107866,85346,3471,0.0,107873,80248,30100,25174.0,...,0.0,0,0.0,16.0,822.135405,1488.0,1395.0,93.0,35.360663,33.150621
14211,19031,20296,95884,80248,30100,25174.0,95885,92363,78433,0.0,...,0.0,1,0.0,20.0,815.308053,1028.0,1028.0,0.0,47.586073,47.586073
14212,18163,19428,94148,92363,78433,0.0,94149,80248,30100,25174.0,...,0.0,0,0.0,20.0,815.308053,867.0,867.0,0.0,56.422703,56.422703
14213,21301,22566,112480,80248,30100,25174.0,112487,85346,3471,0.0,...,0.0,1,0.0,16.0,824.063808,1553.0,1332.0,221.0,37.119991,31.837623


## Pivot and Join Datasets

First Save the Original Long Files

Wagon Data

In [60]:
writer = pd.ExcelWriter(output_dir + r'/wagon_data.xlsx', engine='xlsxwriter')
wagon_data.to_excel(writer, sheet_name='wagon_data', index=False)
writer.save()

Change station_data from long to wide and join to train_data

In [61]:
station_data

,IDTRAIN,IDGARE,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Actual_Arrival,Actual_Departure,Schedule,...,Planned_Departure_Date,Planned_Arrival_Date,Planned_Departure_DOW,Planned_Arrival_DOW,Actual_Departure_Bin,Actual_Arrival_Bin,Origin_Station_Bin,Destination_Station_Bin,BB_Departure_Bin,Full_Journey_Timestamp_Bin
0,255,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,0.0,0.0,1,0,1,0.0
1,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,1,0.0
2,255,85792,2,Bettembourg-frontière,LU,49.467859,6.109153,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,1,0.0
3,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Arrival,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,1,0.0
4,255,87910,3,Hagondange,FR,49.245710,6.162933,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116627,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Arrival,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,0,0.0
116628,22417,87935,9,Metz-Sablon,FR,49.096407,6.166320,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,0,0.0
116629,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,0,0.0
116630,22417,85791,10,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,0,0.0


In [62]:
station_data_unstack = station_data[['IDTRAIN','Timestamp_Order','Station_Name', 'Plan_Timestamp', 'Actual_Timestamp', 'Time_From_Prior_Plan_Mins','Depart_Variance_Mins', 'Arrive_Variance_Mins', 'Travel_Time_Mins', 'Idle_Time_Mins', 'KM_Distance_Event', 'KM_HR_Event']]
station_data_unstack = station_data_unstack.sort_values(['IDTRAIN','Timestamp_Order'])
station_data_unstack = station_data_unstack.set_index(['IDTRAIN','Timestamp_Order'])
station_data_unstack = station_data_unstack.unstack(level=-1)
station_data_unstack.columns = [f"{x}_{y}" for x, y in station_data_unstack.columns.to_flat_index()]
station_data_unstack = station_data_unstack.reset_index().rename(columns={station_data_unstack.index.name:'IDTRAIN'})
train_data = pd.merge(train_data,station_data_unstack,left_on=['IDTRAIN'],right_on=['IDTRAIN'],how='left')

train_data

,IDTRAIN,IDTRAIN_LOT,IDTRAIN_JALON_origin,IDGARE_origin,Code_origin,IDGARE_Parent_origin,IDTRAIN_JALON_destination,IDGARE_destination,Code_destination,IDGARE_Parent_destination,...,KM_HR_Event_14,KM_HR_Event_15,KM_HR_Event_16,KM_HR_Event_17,KM_HR_Event_18,KM_HR_Event_19,KM_HR_Event_20,KM_HR_Event_21,KM_HR_Event_22,KM_HR_Event_23
0,269,269,3619,84831,14108,0.0,3620,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,259,259,3539,80248,30100,25174.0,3540,84831,14108,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,362,362,4363,80248,30100,25174.0,4370,85346,3471,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,274,274,3659,84831,14108,0.0,3660,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,264,264,3579,80248,30100,25174.0,3580,84831,14108,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14210,20739,22004,107866,85346,3471,0.0,107873,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14211,19031,20296,95884,80248,30100,25174.0,95885,92363,78433,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14212,18163,19428,94148,92363,78433,0.0,94149,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14213,21301,22566,112480,80248,30100,25174.0,112487,85346,3471,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Change wagon_data from long to wide and join to train_data

In [63]:
wagon_data

,IDTRAIN_LOT,IDTRAIN_WAGON,IDWAGON,Modele_wagon,Code_wagon,IDGARE_actuelle,Wagon_Order,Wagon_Type,Max_Speed,Wagon_Model_Tare_Weight,Tare_Weight,Wagon_Start_Date
0,7138,138270,NaN,T3000e NIK C30,NaN,NaN,0,DOUBLE,120,35000,NaN,NaN
1,7138,138271,NaN,60',NaN,NaN,1,SIMPLE,120,19800,NaN,NaN
2,7138,138272,NaN,60',NaN,NaN,2,SIMPLE,120,19800,NaN,NaN
3,7138,138273,NaN,60',NaN,NaN,3,SIMPLE,120,19800,NaN,NaN
4,7138,138274,NaN,60',NaN,NaN,4,SIMPLE,120,19800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
226993,22078,578010,170.0,90',33 88 4962 568-0,82517.0,11,DOUBLE,120,27200,27160.0,NaN
226994,22078,578011,144.0,90',33 88 4961 185-4,82517.0,12,DOUBLE,120,27200,27200.0,NaN
226995,22078,578012,165.0,90',33 88 4962 454-3,82517.0,13,DOUBLE,120,27200,27160.0,NaN
226996,22078,578013,167.0,90',33 88 4962 514-4,82517.0,14,DOUBLE,120,27200,27160.0,NaN


In [64]:
wagon_data_unstack = wagon_data[['IDTRAIN_LOT','Wagon_Order','IDWAGON','Modele_wagon','Wagon_Type','Max_Speed','Wagon_Model_Tare_Weight','Tare_Weight','Wagon_Start_Date']]
wagon_data_unstack = wagon_data_unstack.sort_values(['IDTRAIN_LOT','Wagon_Order'])
wagon_data_unstack = wagon_data_unstack.drop_duplicates(subset=['IDTRAIN_LOT','Wagon_Order'])
wagon_data_unstack = wagon_data_unstack.set_index(['IDTRAIN_LOT','Wagon_Order'])
wagon_data_unstack = wagon_data_unstack.unstack(level=-1)
wagon_data_unstack.columns = [f"{x}_{y}" for x, y in wagon_data_unstack.columns.to_flat_index()]
wagon_data_unstack = wagon_data_unstack.reset_index().rename(columns={wagon_data_unstack.index.name:'IDTRAIN_LOT'})
train_data = pd.merge(train_data,wagon_data_unstack,left_on=['IDTRAIN_LOT'],right_on=['IDTRAIN_LOT'],how='left')

# Limit Dataset on Valid Dates and Arcs

In [65]:
#remove trains on the Esch<->Valenton arc
train_data = train_data[train_data['Arc']!='Valenton']
station_data = station_data[station_data['Arc']!='Valeton']

#remove trains on the Barcelona arc per Nathalie -- irregular service
train_data = train_data[train_data['Arc']!='Barcelona']
station_data = station_data[station_data['Arc']!='Barcelona']

#remove trains on the bettembourg<->Mertert arc
train_data = train_data[train_data['Arc']!='Mertert']
station_data = station_data[station_data['Arc']!='Mertert']

In [66]:
#remove data before November 1, 2019 -- Mirabelle start date per Nathalie
#remove data scheduled for the future aka After Apr 06 2021

start_date = datetime.strptime('2019-11-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2021-04-06', '%Y-%m-%d').date()

after_start_date = train_data["Planned_Departure_Date"] >= start_date
before_end_date = train_data["Planned_Departure_Date"] <= end_date
between_two_dates = after_start_date & before_end_date
train_data = train_data.loc[between_two_dates]
train_data = train_data.reset_index(drop=True)

station_data = pd.merge(station_data,train_data['IDTRAIN'],left_on=['IDTRAIN'],right_on=['IDTRAIN'],how='inner')

station_data

,IDTRAIN,IDGARE,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Actual_Arrival,Actual_Departure,Schedule,...,Planned_Departure_Date,Planned_Arrival_Date,Planned_Departure_DOW,Planned_Arrival_DOW,Actual_Departure_Bin,Actual_Arrival_Bin,Origin_Station_Bin,Destination_Station_Bin,BB_Departure_Bin,Full_Journey_Timestamp_Bin
0,6255,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,2019-11-04 22:05:00,Planned_Departure,...,2020-10-04,2020-10-05,Sunday,Monday,1.0,0.0,1,0,1,0.0
1,6255,85791,2,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,2020-09-13,2020-09-14,Sunday,Monday,0.0,0.0,0,0,1,0.0
2,6255,85791,2,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Departure,...,2020-09-20,2020-09-21,Sunday,Monday,0.0,0.0,0,0,1,0.0
3,6255,91845,3,Perrigny,FR,47.290468,5.029242,NaT,NaT,Planned_Arrival,...,2020-09-27,2020-09-28,Sunday,Monday,0.0,0.0,0,0,1,0.0
4,6255,91845,3,Perrigny,FR,47.290468,5.029242,NaT,NaT,Planned_Departure,...,2020-10-04,2020-10-05,Sunday,Monday,0.0,0.0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66345,22403,88656,7,Antwerpen-Noord N,BE,51.290895,4.378313,NaT,NaT,Planned_Arrival,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,1,0.0
66346,22403,88656,7,Antwerpen-Noord N,BE,51.290895,4.378313,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,0.0,0.0,0,0,1,0.0
66347,22403,82688,9999,Antwerp,BE,51.371889,4.276158,NaT,NaT,Planned_Arrival,...,NaT,NaT,NaN,NaN,0.0,0.0,0,1,1,0.0
66348,22407,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,Planned_Departure,...,NaT,NaT,NaN,NaN,NaN,NaN,1,0,1,NaN


In [67]:
### REMOVING OUTLIERS
# find Q1, Q3, and interquartile range for each column
# flag as outliers rows in dataframe that have values outside 1.5*IQR of Q1 and Q3

depart_q1 = train_data['Depart_Variance_Mins'].quantile(q=0.05)
depart_q3 = train_data['Depart_Variance_Mins'].quantile(q=0.95)
arrive_q1 = train_data['Arrive_Variance_Mins'].quantile(q=0.05)
arrive_q3 = train_data['Arrive_Variance_Mins'].quantile(q=0.95)

depart_iqr = train_data['Depart_Variance_Mins'].apply(stats.iqr)
arrive_iqr = train_data['Arrive_Variance_Mins'].apply(stats.iqr)

#calculate 1.5 iqr
depart_q1_iqr = depart_q1-1.5*depart_iqr
depart_q3_iqr = depart_q3+1.5*depart_iqr
arrive_q1_iqr = arrive_q1-1.5*arrive_iqr
arrive_q3_iqr = arrive_q3+1.5*arrive_iqr

print("Depart Q1 Threshold: ",depart_q1)
print("Depart Q3 Threshold: ",depart_q3)
print("Arrive Q1 Threshold: ",arrive_q1)
print("Arrive Q3 Threshold: ",arrive_q3)

#create the outlier filters
outlier_depart_q1 = train_data[train_data['Depart_Variance_Mins']<depart_q1_iqr]
outlier_depart_q3 = train_data[train_data['Depart_Variance_Mins']>depart_q3_iqr]
print(outlier_depart_q3.shape)
outlier_arrive_q1 = train_data[train_data['Arrive_Variance_Mins']<arrive_q1_iqr]
print(outlier_arrive_q1.shape)
outlier_arrive_q3 = train_data[train_data['Arrive_Variance_Mins']>arrive_q3_iqr]
print(outlier_arrive_q3.shape)

print('Number Depart Below Q1 Threshold: ',len(outlier_depart_q1))
print('Number Depart Above Q3 Threshold: ',len(outlier_depart_q3))
print('Number Arrive Below Q1 Threshold: ',len(outlier_arrive_q1))
print('Number Arrive Above Q3 Threshold: ',len(outlier_arrive_q3))

outliers = pd.concat(objs=[outlier_depart_q1,outlier_depart_q3,outlier_arrive_q1,outlier_arrive_q3], axis=0, ignore_index=True)
outliers = outliers.drop_duplicates(subset=['IDTRAIN'], keep='first', ignore_index=True)
outliers = outliers[['IDTRAIN']]
outliers['Outlier_Bin'] = 1
print("Total Outliers: ",len(outliers))

#write a bin field with outlier data
train_data = pd.merge(train_data, outliers, how='left', left_on=['IDTRAIN'], right_on=['IDTRAIN'])
train_data['Outlier_Bin'] = train_data['Outlier_Bin'].fillna(value=0)
train_data.loc[train_data['Cancelled_Train_Bin'] == 1, 'Outlier_Bin'] = np.nan

station_data = pd.merge(station_data,outliers,how='left',left_on=['IDTRAIN'],right_on=['IDTRAIN'])
station_data['Outlier_Bin'] = station_data['Outlier_Bin'].fillna(value=0)
station_data.loc[station_data['Cancelled_Train_Bin'] == 1, 'Outlier_Bin'] = np.nan

#save outlier thresholds to the dataset
train_data['Dep_Outlier_Lower_Threshold'] = depart_q1_iqr
train_data['Dep_Outlier_Upper_Threshold'] = depart_q3_iqr
train_data['Arr_Outlier_Lower_Threshold'] = arrive_q1_iqr
train_data['Arr_Outlier_Upper_Threshold'] = arrive_q3_iqr

station_data['Dep_Outlier_Lower_Threshold'] = depart_q1_iqr
station_data['Dep_Outlier_Upper_Threshold'] = depart_q3_iqr
station_data['Arr_Outlier_Lower_Threshold'] = arrive_q1_iqr
station_data['Arr_Outlier_Upper_Threshold'] = arrive_q3_iqr

Depart Q1 Threshold:  -73.0
Depart Q3 Threshold:  209.64999999999964
Arrive Q1 Threshold:  -116.0
Arrive Q3 Threshold:  1092.5999999999995
(210, 566)
(207, 566)
(210, 566)
Number Depart Below Q1 Threshold:  204
Number Depart Above Q3 Threshold:  210
Number Arrive Below Q1 Threshold:  207
Number Arrive Above Q3 Threshold:  210
Total Outliers:  720


In [68]:
#write processing timestamp

dateTimeObj = datetime.now()
train_data['Processing_Date'] = dateTimeObj
station_data['Processing_Date'] = dateTimeObj

In [69]:
train_data

,IDTRAIN,IDTRAIN_LOT,IDTRAIN_JALON_origin,IDGARE_origin,Code_origin,IDGARE_Parent_origin,IDTRAIN_JALON_destination,IDGARE_destination,Code_destination,IDGARE_Parent_destination,...,Wagon_Start_Date_32,Wagon_Start_Date_33,Wagon_Start_Date_34,Wagon_Start_Date_35,Outlier_Bin,Dep_Outlier_Lower_Threshold,Dep_Outlier_Upper_Threshold,Arr_Outlier_Lower_Threshold,Arr_Outlier_Upper_Threshold,Processing_Date
0,7223,8203,36968,82517,91508,0.0,36974,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
1,7124,8031,36249,80248,30100,25174.0,36256,85346,3471,0.0,...,NaN,NaN,NaN,NaN,0.0,-73.0,209.65,-116.0,1092.6,2021-05-19 12:58:04.234856
2,7438,8444,38260,80248,30100,25174.0,38261,92363,78433,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
3,6798,7705,34733,80248,30100,25174.0,34734,92363,78433,0.0,...,NaN,NaN,NaN,NaN,1.0,-73.0,209.65,-116.0,1092.6,2021-05-19 12:58:04.234856
4,6498,7405,33977,80248,30100,25174.0,33980,84831,14108,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7815,22268,23533,119619,82688,24953,0.0,119624,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
7816,18222,19487,94266,92363,78433,0.0,94267,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,0.0,-73.0,209.65,NaN,NaN,2021-05-19 12:58:04.234856
7817,22343,23608,120213,85346,3471,0.0,120221,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,0.0,-73.0,209.65,NaN,NaN,2021-05-19 12:58:04.234856
7818,22210,23475,119271,83760,72920,0.0,119276,80248,30100,25174.0,...,NaN,NaN,NaN,NaN,0.0,-73.0,209.65,-116.0,1092.6,2021-05-19 12:58:04.234856


In [70]:
station_data

,IDTRAIN,IDGARE,Station_Order,Station_Name,Station_Country,Station_Latitude,Station_Longitude,Actual_Arrival,Actual_Departure,Schedule,...,Origin_Station_Bin,Destination_Station_Bin,BB_Departure_Bin,Full_Journey_Timestamp_Bin,Outlier_Bin,Dep_Outlier_Lower_Threshold,Dep_Outlier_Upper_Threshold,Arr_Outlier_Lower_Threshold,Arr_Outlier_Upper_Threshold,Processing_Date
0,6255,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,2019-11-04 22:05:00,Planned_Departure,...,1,0,1,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
1,6255,85791,2,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Arrival,...,0,0,1,0.0,0.0,-73.0,209.65,-116.0,1092.6,2021-05-19 12:58:04.234856
2,6255,85791,2,Zoufftgen-Frontiere,FR,49.467859,6.109153,NaT,NaT,Planned_Departure,...,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
3,6255,91845,3,Perrigny,FR,47.290468,5.029242,NaT,NaT,Planned_Arrival,...,0,0,1,0.0,0.0,-73.0,209.65,-116.0,1092.6,2021-05-19 12:58:04.234856
4,6255,91845,3,Perrigny,FR,47.290468,5.029242,NaT,NaT,Planned_Departure,...,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66345,22403,88656,7,Antwerpen-Noord N,BE,51.290895,4.378313,NaT,NaT,Planned_Arrival,...,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
66346,22403,88656,7,Antwerpen-Noord N,BE,51.290895,4.378313,NaT,NaT,Planned_Departure,...,0,0,1,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
66347,22403,82688,9999,Antwerp,BE,51.371889,4.276158,NaT,NaT,Planned_Arrival,...,0,1,1,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856
66348,22407,80248,1,Bettembourg,LU,49.493889,6.108889,NaT,NaT,Planned_Departure,...,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-19 12:58:04.234856


# Final Dataset Files

Write files to excel

In [71]:
writer = pd.ExcelWriter(output_dir + r'/train_data.xlsx', engine='xlsxwriter')
train_data.to_excel(writer, sheet_name='train_data', index=False)
writer.save()

In [72]:
writer = pd.ExcelWriter(output_dir + r'/station_data.xlsx', engine='xlsxwriter')
station_data.to_excel(writer, sheet_name='station_data', index=False)
writer.save()

In [73]:
writer = pd.ExcelWriter(output_dir + r'/train_position.xlsx', engine='xlsxwriter')
train_position.to_excel(writer, sheet_name='train_position', index=False)
writer.save()

Write to Google Sheets

In [74]:
#send data to google sheets

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(output_dir + r'/cfl-thesis-60585bc96a06.json', scopes=scope)
gc = gspread.authorize(credentials)
google_sheets_limit = 5000000

#train_data
try:
    if train_data.size <= google_sheets_limit:
        spreadsheet_key ='11P9kmFV5JiU8MEmzeAkt1EfSHPIUViRVcwi2_FIRQdQ'
        wks_name = 'train_data'
        d2g.upload(train_data, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
        print('train_data upload success.')
    else: 
        print('train data is too big. Get rid of some data.')
except: print('There was a problem uploading the file.')

train_data upload success.


In [75]:
#send data to google sheets

#station_data
try:
    if station_data.size <= google_sheets_limit:
        spreadsheet_key ='1oKZjhWOcH9dfOer0eOd4kqBtwKMZxM4t6Ah1q9u7g8g'
        wks_name = 'station_data'
        d2g.upload(station_data, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
        print('station_data upload success.')
    else:
        print('station_data is too big. Get rid of some data.')
except:
    print('There was a problem uploading the file.')

station_data is too big. Get rid of some data.


Write to xlsb - WIP

In [76]:
#this doesn't seem to work. writes a file infinitely that can't be opened.

#wb = open_workbook(output_dir + r'/CFL_Data.xlsb', 'w')
#ws = wb.create_sheet('train_data')
#try:
#    a = train_data
#    ws.write_table(a)
#finally:
#    ws.close()  # Ensure sheets are closed before writing any other sheets.
#ws = wb.create_sheet('station_data')
#try:
#    b = station_data
#    ws.write_table(b)
#finally:
#    ws.close()
#wb.close()  # Ensure all sheets are closed before closing the workbook.''''''

Write to Tableau Hyper

In [77]:
cfl_data = {"train_data": train_data, "station_data": station_data, "train_position": train_position}
pantab.frames_to_hyper(cfl_data, output_dir + r'/cfl_data.hyper')

TypeError: Conversion of 'UInt32' dtypes not supported!